In [23]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from importlib import reload
import sys 
import pandas as pd
import numpy as np
import sys 
sys.path.append('/Users/Santiago PC/Documents/Photometry/Quarter1/time_triggered_average/')
import photometry_3
import time_triggered_average
get_ipython().magic('matplotlib qt5')
import matplotlib.pyplot as plt
import scipy.ndimage
from tqdm import tqdm
from sklearn import preprocessing 

In [24]:
def plot_averages(df, channel_to_align, Fs,  x_lims_list, y_lims_list):
    
    time_base = np.divide(df.index.values, Fs)
    
    plt.subplot(3,1,1)
    plt.plot(time_base, df[channel_to_align], 'b')
    plt.xlabel('time (seconds)')
    plt.ylabel(channel_to_align)
    plt.ylim(y_lims_list[0][0], y_lims_list[0][1])
    #plt.xlim(50000, 150000)
    plt.subplot(3,1,2)
    plt.plot(time_base, df['channel_1_demod_F_F0'], 'g')
    plt.xlabel('time (seconds)')
    plt.ylabel('F/F0 488')
    plt.ylim(y_lims_list[1][0], y_lims_list[1][1])
    #plt.xlim(50000, 150000)
    plt.subplot(3,1,3)
    plt.plot(time_base, df['channel_2_demod_F_F0'])
    plt.xlabel('time (seconds)')
    plt.ylabel('F/F0 405')
    plt.ylim(y_lims_list[2][0], y_lims_list[2][1])
    #plt.xlim(50000, 150000)
    plt.show()
    return(True)

def plot_individual_events(df, channel_to_align, Fs,  x_lims_list, y_lims_list):
    
    time_base = np.divide(df.index.values, Fs)
    
    plt.subplot(2,1,1)
    for event in df[channel_to_align].columns:   
        plt.plot(df[channel_to_align][event])
    plt.xlabel('time (seconds)')
    plt.ylabel(channel_to_align)
    plt.ylim(y_lims_list[0][0], y_lims_list[0][1])
    #plt.xlim(50000, 150000)
    plt.subplot(2,1,2)
    for event in df['channel_1_demod_F_F0'].columns:   
        plt.plot(df['channel_1_demod_F_F0'][event])
    plt.xlabel('time (seconds)')
    plt.ylabel('F/F0 488')
    plt.ylim(y_lims_list[1][0], y_lims_list[1][1])
    plt.show()
    return(True)

In [46]:
reload(photometry_3)
reload(time_triggered_average)

<module 'time_triggered_average' from '/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/photometry_analysis/time_triggered_average.py'>

In [26]:
#pd.read_hdf('/Users/johnmarshall/Documents/Analysis/PhotometryData/2019/07132019/19713011_F_F0behavior_added.h5')

In [27]:
f_name = '/Users/johnmarshall/Documents/Analysis/PhotometryData/2019/08072019/SC_08072019/19807000background_subtracted_F_F0behavior_addedaccel.h5'
df_from_file = photometry_3.photometry_df_from_file(f_name)

In [28]:
df_from_file.class_df.head()

time  encoder_windowed  fluorescence_channel_1_windowed  \
sweeps   index                                                               
sweep001 16846  16.6846          4.913940                         3.298340   
         16847  16.6847          4.913330                         3.966980   
         16848  16.6848          4.915161                         4.398193   
         16849  16.6849          4.916687                         4.578857   
         16850  16.6850          4.915771                         4.463501   

                channel_1_demod  channel_2_demod  channel_1_demod_F_F0  \
sweeps   index                                                           
sweep001 16846         0.079159         0.185719              1.000596   
         16847         0.079158         0.185768              1.000596   
         16848         0.079157         0.185816              1.000596   
         16849         0.079156         0.185863              1.000596   
         16850         0.079154         0.185909              1.000596   

                channel_2_demod_F_F0  video_times_from_sync       frame  \
sweeps   index                                                            
sweep001 16846              0.991251              16.670954  500.000000   
         16847              0.991251              16.671054  500.002999   
         16848              0.991251              16.671154  500.005998   
         16849              0.991251              16.671254  500.008997   
         16850              0.991251              16.671354  500.011997   

                Acceleration  
sweeps   index                
sweep001 16846     -0.169425  
         16847     -0.169425  
         16848     -0.169425  
         16849     -0.169425  
         16850     -0.169425

In [29]:
channel_to_align = 'Acceleration'

In [30]:
df_from_file.class_df.head()

time  encoder_windowed  fluorescence_channel_1_windowed  \
sweeps   index                                                               
sweep001 16846  16.6846          4.913940                         3.298340   
         16847  16.6847          4.913330                         3.966980   
         16848  16.6848          4.915161                         4.398193   
         16849  16.6849          4.916687                         4.578857   
         16850  16.6850          4.915771                         4.463501   

                channel_1_demod  channel_2_demod  channel_1_demod_F_F0  \
sweeps   index                                                           
sweep001 16846         0.079159         0.185719              1.000596   
         16847         0.079158         0.185768              1.000596   
         16848         0.079157         0.185816              1.000596   
         16849         0.079156         0.185863              1.000596   
         16850         0.079154         0.185909              1.000596   

                channel_2_demod_F_F0  video_times_from_sync       frame  \
sweeps   index                                                            
sweep001 16846              0.991251              16.670954  500.000000   
         16847              0.991251              16.671054  500.002999   
         16848              0.991251              16.671154  500.005998   
         16849              0.991251              16.671254  500.008997   
         16850              0.991251              16.671354  500.011997   

                Acceleration  
sweeps   index                
sweep001 16846     -0.169425  
         16847     -0.169425  
         16848     -0.169425  
         16849     -0.169425  
         16850     -0.169425

In [31]:
plt.plot(df_from_file.class_df.loc['sweep007']['Acceleration'].values)
plt.show()

In [32]:
#df_from_file.class_df
#display spreadsheet w data

In [33]:
if channel_to_align == 'Acceleration':
    mean_=np.nanmean(df_from_file.class_df[channel_to_align].values)
    std_=np.nanstd(df_from_file.class_df[channel_to_align].values)
    threshold = mean_ + 6*std_
    print(mean_)
    print(std_)
    print(threshold)
    time_threshold = 0
    
if channel_to_align == 'Digging Manual':
    threshold = 0.5
    time_threshold = 0

-0.058204011024027155
130.31207249105472
781.8142309353043


In [34]:
# return regions in traces "triggered" around an amplitude threshold in a given trace 
regions = time_triggered_average.align_on_trigger(df_from_file.class_df, 10000, 
                                                     channel_to_align, 
                                                     ['channel_1_demod_F_F0', 'channel_2_demod_F_F0'],
                                                     threshold, 'down', 
                                                     time_threshold, 
                                                     5, (0, 0))

50000


In [52]:
#average the triggered regions across channels
unsorted_regions = time_triggered_average.return_events(regions)
unsorted_regions_mean = unsorted_regions.mean(axis=1, level=0)

In [53]:
unsorted_regions.head()

channel_1_demod_F_F0                                                    \
              sweep005                                                     
                220581    229250    232579    401909    403239    405239   
0             1.070336  0.983642  0.999885  0.983931  0.981019  0.995647   
1             1.070352  0.983637  1.000003  0.983917  0.980946  0.995712   
2             1.070369  0.983628  1.000122  0.983903  0.980876  0.995774   
3             1.070388  0.983615  1.000241  0.983888  0.980809  0.995833   
4             1.070407  0.983598  1.000361  0.983873  0.980745  0.995888   

                                           ... Acceleration                    \
   sweep002            sweep001            ...     sweep007                     
     482921    494919    284629    644953  ...       305929   437599   443270   
0  1.000616  1.035890  1.028292  0.960145  ...      26.9432 -363.473 -41.9422   
1  1.000686  1.035984  1.028215  0.960189  ...      26.9432 -363.473 -41.9422   
2  1.000752  1.036074  1.028136  0.960239  ...      26.9432 -363.473 -41.9422   
3  1.000815  1.036161  1.028055  0.960293  ...      26.9432 -363.473 -41.9422   
4  1.000874  1.036246  1.027972  0.960352  ...      26.9432 -363.473 -41.9422   

                                                                  
           sweep014                                               
    446600   97311    99652    101984   105316   138004   141336  
0 -3.13263 -1.66046 -25.8759 -18.1398  74.9142 -196.332 -50.4693  
1 -3.13263 -1.66046 -25.8759 -18.1398  74.9142 -196.332 -50.4693  
2 -3.13263 -1.66046 -25.8759 -18.1398  74.9142 -196.332 -50.4693  
3 -3.13263 -1.66046 -25.8759 -18.1398  74.9142 -196.332 -50.4693  
4 -3.13263 -1.66046 -25.8759 -18.1398  74.9142 -196.332 -50.4693  

[5 rows x 201 columns]

In [38]:
#unsorted_regions['channel_2_demod_F_F0']['sweep009'][81309]

In [39]:
plt.plot(unsorted_regions['channel_2_demod_F_F0']['sweep005'][220581])
#plt.plot(unsorted_regions['Acceleration']['sweep009'][275965])
plt.show()

In [66]:
plt.plot(unsorted_regions['channel_1_demod_F_F0'])
plt.show()

In [67]:
plt.plot(unsorted_regions['channel_2_demod_F_F0'].mean(axis=1))
plt.plot(unsorted_regions['channel_1_demod_F_F0'].mean(axis=1))
plt.show()


In [49]:
plt.plot(unsorted_regions['Acceleration'].mean(axis=1))
plt.show()

In [43]:
threshold

781.8142309353043

In [61]:
#sort events by additional criteria 
sorted_events = time_triggered_average.return_sorted_events(unsorted_regions, channel_to_align , 'before', threshold, (0,20000))

100%|██████████| 67/67 [00:00<00:00, 313.99it/s]


In [62]:
sorted_events.head()


channel_1_demod_F_F0                                                    \
              sweep005                                sweep002  sweep001   
                401909    405239    220581    403239    482921    644953   
0             0.983931  0.995647  1.070336  0.981019  1.000616  0.960145   
1             0.983917  0.995712  1.070352  0.980946  1.000686  0.960189   
2             0.983903  0.995774  1.070369  0.980876  1.000752  0.960239   
3             0.983888  0.995833  1.070388  0.980809  1.000815  0.960293   
4             0.983873  0.995888  1.070407  0.980745  1.000874  0.960352   

                                           ... Acceleration                    \
             sweep003                      ...     sweep018          sweep016   
     284629    358406    479435    263470  ...       266325   307997   1269     
0  1.028292  0.999336  0.985459  0.985952  ...     -25.8759 -50.4693      NaN   
1  1.028215  0.999207  0.985352  0.986095  ...     -25.8759 -50.4693      NaN   
2  1.028136  0.999075  0.985249  0.986236  ...     -25.8759 -50.4693      NaN   
3  1.028055  0.998940  0.985151  0.986373  ...     -25.8759 -50.4693      NaN   
4  1.027972  0.998804  0.985059  0.986507  ...     -25.8759 -50.4693      NaN   

                                                                  
  sweep017 sweep007          sweep014                             
    191940   305929   437599   141336   99652    138004   97311   
0  372.354  26.9432 -363.473 -50.4693 -25.8759 -196.332 -1.66046  
1  372.354  26.9432 -363.473 -50.4693 -25.8759 -196.332 -1.66046  
2  372.354  26.9432 -363.473 -50.4693 -25.8759 -196.332 -1.66046  
3  372.354  26.9432 -363.473 -50.4693 -25.8759 -196.332 -1.66046  
4  372.354  26.9432 -363.473 -50.4693 -25.8759 -196.332 -1.66046  

[5 rows x 129 columns]

In [ ]:
#save results to hdf file 
unsorted_regions.to_hdf(f_name.rstrip('.h5') + channel_to_align + 'unsorted.h5', key='df', format='table')
sorted_events.to_hdf(f_name.rstrip('.h5') + channel_to_align + 'sorted_before.h5', key='df')

In [ ]:
#load sorted events dataframe from file 
sorted_events = pd.read_hdf('/Users/johnmarshall/Documents/Analysis/PhotometryData/2019/08072019/SC_08072019/19807000background_subtracted_F_F0behavior_addedaccelAccelerationsorted_before.h5')

In [71]:
sorted_events_sweeps_removed = sorted_events.droplevel(level=1, axis=1)

In [72]:
plt.plot(sorted_events_sweeps_removed['Acceleration'])
plt.show()

In [73]:
plt.plot(sorted_events_sweeps_removed['channel_2_demod_F_F0'].mean(axis=1)[15000:35000])
plt.plot(sorted_events_sweeps_removed['channel_1_demod_F_F0'].mean(axis=1)[15000:35000])
plt.show()

In [74]:
sorted_events_sweeps_removed['channel_1_demod_F_F0'].head()

,401909,405239,220581,403239,482921,644953,284629,358406,479435,263470,...,266325,307997,1269,191940,305929,437599,141336,99652,138004,97311
0,0.983931,0.995647,1.070336,0.981019,1.000616,0.960145,1.028292,0.999336,0.985459,0.985952,...,0.993896,1.003232,NaN,0.991339,1.001691,1.004476,1.019551,0.958780,1.016350,0.957745
1,0.983917,0.995712,1.070352,0.980946,1.000686,0.960189,1.028215,0.999207,0.985352,0.986095,...,0.994066,1.003186,NaN,0.991368,1.001850,1.004489,1.019728,0.958782,1.016388,0.957714
2,0.983903,0.995774,1.070369,0.980876,1.000752,0.960239,1.028136,0.999075,0.985249,0.986236,...,0.994237,1.003142,NaN,0.991399,1.002006,1.004498,1.019903,0.958782,1.016423,0.957681
3,0.983888,0.995833,1.070388,0.980809,1.000815,0.960293,1.028055,0.998940,0.985151,0.986373,...,0.994409,1.003099,NaN,0.991430,1.002160,1.004501,1.020076,0.958782,1.016456,0.957649
4,0.983873,0.995888,1.070407,0.980745,1.000874,0.960352,1.027972,0.998804,0.985059,0.986507,...,0.994583,1.003059,NaN,0.991462,1.002310,1.004499,1.020248,0.958781,1.016488,0.957616


In [75]:
plt.plot(sorted_events_sweeps_removed['channel_1_demod_F_F0'][15000:35000])
plt.show()

In [ ]:
sorted_events_cropped = sorted_events_sweeps_removed['channel_1_demod_F_F0'][15000:35000]

In [76]:
plt.plot(sorted_events_sweeps_removed.mean(axis=1))
plt.show()

In [77]:
def range_scale_numpy_array(inp_array):
    inp_array_std = (inp_array - np.min(inp_array)) / (np.max(inp_array) - np.min(inp_array))
    return(inp_array_std)

In [79]:
range_scaled = {}
for event in list(sorted_events_sweeps_removed.columns):
    range_scaled[event] = range_scale_numpy_array(sorted_events_sweeps_removed[event])
range_scaled = pd.DataFrame(range_scaled)

In [83]:
plt.plot(sorted_events_sweeps_removed['channel_1_demod_F_F0'].mean(axis=1))
plt.show()

In [84]:
plt.plot(range_scale_numpy_array(sorted_events_sweeps_removed['channel_1_demod_F_F0'].mean(axis=1).values))
plt.show()

In [81]:
plt.plot(range_scaled['channel_1_demod_F_F0'].mean(axis=1))
plt.show()

In [ ]:
plt.plot(range_scaled[399315])
plt.show()

In [ ]:
plt.plot(range_scaled[399315])
plt.show()

In [ ]:
plt.plot(range_scaled)

In [ ]:
#plot average trace from sorted regions 
df_from_file.Fs
plot_averages(means_by_channel, channel_to_align, df_from_file.Fs, [(),(),()], [(-.1, 1.2),(.98, 1.01),(.98, 1.025)])

In [ ]:
plt.subplot(2,1,1)
plt.plot(unsorted_regions['Grooming'])
plt.subplot(2,1,2)
plt.plot(unsorted_regions['channel_1_demod_F_F0'])
plt.show()

In [ ]:
unsorted_regions['channel_1_demod_F_F0'].loc[50000:60000].max().values

In [ ]:
max_during_grooming = unsorted_regions['channel_1_demod_F_F0'].loc[50000:60000].max().values

In [ ]:
max_before_grooming = unsorted_regions['channel_1_demod_F_F0'].loc[40000:50000].max().values

In [ ]:
plt.scatter(np.ones(len(max_during_grooming)), max_during_grooming)

In [ ]:
plt.scatter(np.ones(len(max_before_grooming)), max_before_grooming)

In [ ]:
max_before_grooming.max()

In [ ]:
max_during_grooming.max()

In [ ]:
plt.hist([max_before_grooming, max_during_grooming])
plt.show()
plt.suptitle('AAV_Synflex_004 D1')
plt.xlabel('Fluoresence')
plt.ylabel('Frequency')

In [ ]:
max_before = [1.011622661,1.028583386]
max_during = [1.014063491,1.025865516]

In [ ]:
unsorted_regions['channel_1_demod_F_F0'].loc[50000:60000].max()

In [ ]:
#plot individual traces of sorted events
plot_individual_events(sorted_events, channel_to_align, df_from_file.Fs, [(),(),()], [(-.1, 1.2),(.98, 1.025),(.98, 1.025)])

In [ ]:
#get peak of events in individual traces
peak_df = pd.DataFrame(sorted_events[30000:50000].max(axis=0), columns = ['peak amplitude']).transpose()

In [ ]:
#calculate latency to peak 
latency_samples_of_peak_df = pd.DataFrame(sorted_events[30000:50000].idxmax(axis=0), columns = ['peak latency']).transpose()
#convert to seconds
time_of_peak_df = latency_samples_of_peak_df.apply(lambda x: (50001-x)/df_from_file.Fs)

In [ ]:
# fiter photometry signal but leave behavior channels 
dfs = []
names = []
for channel in ['channel_1_demod_F_F0', 'channel_2_demod_F_F0']:
    by_channel = {}
    for event in sorted_events['channel_1_demod_F_F0'].columns.values:
        by_channel[event] = sorted_events[channel][event].apply((lambda x: photometry_3.gaussian_filter_signal(x, 200)))
    dfs.append(pd.DataFrame(by_channel))
    names.append(channel)

dfs.append(sorted_events['Acceleration'])
names.append('Acceleration')

filt_df = pd.concat(dfs, keys=names, axis=1)

In [ ]:
plot_individual_events(filt_df, channel_to_align, df_from_file.Fs, [(),(),()], [(-.1, 2000),(.90, 1.05),(.95, 1.05)])

In [ ]:
dfs_output = pd.concat([peak_df, time_of_peak_df])

In [ ]:
dfs_output

In [ ]:
plt.scatter(dfs_output['channel_1_demod_F_F0'].loc['peak latency'].values, dfs_output[channel_to_align].loc['peak amplitude'].values)

In [ ]:

dfs_output.to_hdf(directory+f_name.rstrip('behavior_added.h5')+'_'+str(channel_to_align)+'_bouts_sorted_before_peak.h5', key='df')

In [ ]:
#for grooming analysis
# does the peak F_F0 proximal to grooming onset correlate with bout length

# get bout lengths
df = sorted_events
bout_lengths = {}
for index in df[channel_to_align].columns:  
    out = time_triggered_average.measure_bout_lengths(df[channel_to_align][index], start_index=40000, threshold=.5)
    if out[0][0] == int(len(df[channel_to_align][index])/2)+1:
        bout_lengths[index] = out[1][0]
#list(bout_lengths.items())

In [ ]:
# from bout length dictionary get the peak in the F_F0 trace 
F_F0_peaks = {}
for item in list(bout_lengths.items()):
    print(item)
    middle_index = int(len(df['channel_1_demod_F_F0'][item[0]])/2)
    start_index = middle_index - 30000
    signal_filt = photometry_3.gaussian_filter_signal(df['channel_1_demod_F_F0'][item[0]][start_index:(middle_index+item[1])], 200)
    F_F0_peaks[item[0]] = np.max(signal_filt)
    plt.plot(signal_filt)
plt.show()


df_out = pd.concat([pd.DataFrame(list(bout_lengths.values()), index=list(bout_lengths.keys()), columns=['bout_lengths_samples']),
          pd.DataFrame(list(F_F0_peaks.values()), index=list(F_F0_peaks.keys()), columns=['channel_1_F_F0_peak'])], axis=1)

In [ ]:
df_out.plot.scatter('channel_1_F_F0_peak', 'bout_lengths_samples', title='Mouse: AAV_SYN_005 File: ' + f_name.rstrip('behavior_added.h5'))

In [ ]:
# renormalize to a baseline before grooming onset
re_normed_traces = {}
F_F0_peaks_normed_before_onset = {}
for item in list(bout_lengths.items()):
    print(item)
    middle_index = int(len(df['channel_1_demod_F_F0'][item[0]])/2)
    start_index = middle_index-30000
    signal_filt = photometry_3.gaussian_filter_signal(df['channel_1_demod_F_F0'][item[0]][start_index:(middle_index+item[1])], 200)
    
    signal_filt_normed = np.array([((sample-np.mean(signal_filt[0:10000]))/np.mean(signal_filt[0:10000])) for sample in signal_filt])
    
    F_F0_peaks_normed_before_onset[item[0]] = np.max(signal_filt_normed)
    to_pad = 80000-len(signal_filt)
    re_normed_traces[item[0]] = np.pad(signal_filt_normed,(0,to_pad), 'constant', constant_values=np.nan)
    plt.plot(signal_filt_normed)
plt.show()

df_out = pd.concat([pd.DataFrame(list(bout_lengths.values()), index=list(bout_lengths.keys()), columns=['bout_lengths_samples']),
         pd.DataFrame(list(F_F0_peaks.values()), index=list(F_F0_peaks.keys()), columns=['channel_1_F_F0_peak']),
         pd.DataFrame(list(F_F0_peaks_normed_before_onset.values()), index=list(F_F0_peaks_normed_before_onset.keys()), columns=['channel_1_F_F0_peak_normed'])],
                   axis=1
                  )

In [ ]:
df_traces = pd.DataFrame(re_normed_traces, index=np.linspace(0, len(list(re_normed_traces.items())[0][1]), len(list(re_normed_traces.items())[0][1])))

In [ ]:
plt.plot(df_traces.mean(axis=1))
plt.show()

In [ ]:
df_out.plot.scatter('channel_1_F_F0_peak_normed', 'bout_lengths_samples', title='Mouse: AAV_SYN_004 File: ' + f_name.rstrip('behavior_added.h5'))

In [ ]:
df_out.to_hdf(directory+f_name.rstrip('behavior_added.h5')+'grooming_bouts_sorted_for_plotting.h5', key='df')

In [ ]:
df_out